I have trained total of 7 models i.e 2 Random forest classifier, 2 Xgboost classifier , 
2 shallow Neural network(one each on training with and without validation set)
and 1 deep neural network

In [1]:
import pandas as pd

from sklearn.ensemble.forest import RandomForestClassifier 
 
import numpy as np

import re 

import os

from collections import defaultdict

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix,classification_report,f1_score,accuracy_score

import bisect

import copy

from xgboost import XGBClassifier

from hyperopt import hp

import numpy as np

from sklearn.metrics import mean_squared_error

import hyperopt

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import xgboost as xgb

from xgboost.sklearn import XGBRegressor

from sklearn.metrics import accuracy_score

from sklearn import metrics

from hyperopt import space_eval

from sklearn.metrics import confusion_matrix,classification_report,f1_score,accuracy_score

from xgboost import XGBClassifier

from sklearn.metrics import mean_squared_error




/Users/shrutijain/Applications/Python/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
train_path='/Users/shrutijain/Downloads/Data Scientist _SE_assignment/train/'
all_classes = os.walk(train_path)

In [3]:
test_path='/Users/shrutijain/Downloads/Data Scientist _SE_assignment/test/'
all_classes_test = os.walk(test_path)

In [4]:
def get_file_text(file_path):
    '''
    

    Parameters
    ----------
    file_path : TYPE
        DESCRIPTION.

    Returns
    -------
    text.

    '''
    text=''
    return text


In [5]:
def get_folder_df(folder_path):
    '''

    Parameters
    ----------
    folder_path : TYPE
        DESCRIPTION.

    Returns
    -------
    text.

    '''
    all_classes = os.walk(folder_path)
    
    data_dict={}
    
    for class_ in all_classes:
        
        root_folder = os.path.split(class_[0])[-1]
        
        data_dict[root_folder]={}
        
        for file in class_[2]:
            
            
            if (file.isdigit):
                
                data_dict[root_folder][file]=open(os.path.join(class_[0], file),'rb').read()
        
    reform = {(outerKey, innerKey): [values] for outerKey, innerDict in data_dict.items() for innerKey, values in innerDict.items()}
    
    df=pd.DataFrame.from_dict(reform,orient='index')
    
    df[0]=df[0].str.decode("utf-8",errors='ignore')
    
    df.index = pd.MultiIndex.from_tuples(df.index)
    
    df.reset_index(inplace=True)
    
    df.columns=['classes','file','text']
    
    df=df[df.groupby('classes').classes.transform('count')>1].copy().reset_index(drop=True)
    
    return df



In [6]:
class CustomError(Exception):
    pass


def data_processing(df):
    
    all_text=[]
    
    for x in df.text:
        
        attribute={}
        
        
        
        res_list = x.rstrip().split('\n')
        
        text=[]
        
        for y in res_list:
           
            
            coln_index=y.find(':')
            
            if coln_index>0:
                
                label=y[0:coln_index]
                
                value=y[coln_index+1:]
                
                try:
                    
                    if (value.strip()=='') or (label.strip()==''):
                        
                        raise CustomError()
                    
                    attribute[label.lower().strip()]=value.lower().strip()
                    
                    
                except CustomError:
                    
                    text.append(y)
                    
                    continue
                
            else:
                text.append(y)
                
        other_text='\n'.join(text)
        
        attribute['other_text']=other_text
        
        abc=copy.deepcopy(attribute)
        
        all_text.append(abc)
        
    return all_text


In [7]:
def remove_unintented_labels(lst_dct):
    
    key_labels = [j for i in lst_dct for j in i.keys()]

    dct = defaultdict(int)
    
    for key in key_labels:
      dct[key] += 1
    
    #sorted=dct.reverse()
        
    sorted_dct=dict(sorted(dct.items(),reverse=True, key=lambda item: item[1]))
    
    dict_select_keys = {key:val for key, val in sorted_dct.items() if val >= len(lst_dct)*0.1}
    
    all_text_copy=[]
                            
    for row in lst_dct:
        
        row1 = {sel_key: row[sel_key] for sel_key in (set(row.keys()).intersection(set(dict_select_keys.keys()))) }
        
        other_text_lst=['{}:{}'.format(un_sel,row[un_sel]) for un_sel in set(row.keys() -set(dict_select_keys.keys()))]
        
        row1['other_text']=row1['other_text']+'\n'.join(other_text_lst)
        
        all_text_copy.append(row1.copy())
        
    return all_text_copy

In [8]:
def create_df(path_):
    
    df = get_folder_df(path_)    
    
    text_attribute = data_processing(df)
    
    text_attribute=remove_unintented_labels(text_attribute)
    
    final_df = pd.DataFrame(text_attribute) 

    #final_df = final_df.fillna('')

    final_df["Class"] = ""

    final_df['Class']=df['classes']
    
    final_df['all_text']=df['text']

    return final_df

In [9]:
def interpolate_line_lenght(actual_df):
    
    actual_df.lines = actual_df['lines'].replace('',pd.np.nan)
    
    actual_df['len']=actual_df.all_text.apply(lambda x:len(str(x)))
    
    no_nans=actual_df.dropna(subset=['lines'])
    
    no_nans=no_nans[no_nans.lines.str.isdigit()]
    
    no_nans.lines=no_nans.lines.astype(int)
    
    len_to_line=sum(no_nans.len)/sum(no_nans.lines)
    
    actual_df.lines=np.where(actual_df.lines.isna(),(actual_df.len/len_to_line).astype(int),actual_df.lines)
    
    actual_df.lines=np.where(~actual_df.lines.astype(str).str.isdigit(),(actual_df.len/len_to_line).astype(int),actual_df.lines)
    
    actual_df.lines=actual_df.lines.astype(int)
    
    return actual_df

In [10]:
def tfidf(df,column_name):
    
    vectorizer=TfidfVectorizer(stop_words=['<p>','</p>'],max_df=0.2,min_df=20)

    tf_train=vectorizer.fit_transform(df[column_name])

    tfidf_=pd.DataFrame(tf_train.toarray(),columns=[column_name+'_'+ x for x in vectorizer.get_feature_names()])
    
    return tfidf_,vectorizer

In [11]:
train_df=create_df(path_=train_path)

train_df=interpolate_line_lenght(train_df)

train_df.fillna('',inplace=True)

/Users/shrutijain/Applications/Python/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
X=train_df[['nntp-posting-host', 'other_text', 'from', 'subject', 'organization',
       'lines','reply-to','distribution','all_text']]    

y=train_df['Class']

    
X_train, X_val,y_train, y_val=train_test_split (X,y, random_state=0,stratify=y)

In [13]:

first_tfidf1,v1=tfidf(X_train,'nntp-posting-host')

sec_tfidf,v2=tfidf(X_train,'other_text')

third_tfidf,v3=tfidf(X_train,'from')

fourth_tfidf,v4=tfidf(X_train,'subject')

fifth_tfidf,v5=tfidf(X_train,'organization')

sixth_tfidf,v6=tfidf(X_train,'reply-to')

seventh_tfidf,v7=tfidf(X_train,'distribution')

train_features=pd.concat([ first_tfidf1,sec_tfidf,third_tfidf,fourth_tfidf,fifth_tfidf,sixth_tfidf,seventh_tfidf,X_train[['lines']].reset_index(drop=True)],axis=1)


 

In [14]:
model = XGBClassifier()

model.fit(train_features,y_train)

imp_feats=pd.Series(model.feature_importances_,index=train_features.columns)



In [20]:
imp_features_xgb=imp_feats[imp_feats>0.003]

In [101]:
imp_features_xgb.sort_values(ascending=False)

other_text_dod          0.012516
other_text_gun          0.011503
other_text_car          0.010960
other_text_space        0.009845
subject_sale            0.009407
                          ...   
other_text_via          0.003235
other_text_msg          0.003231
other_text_troops       0.003153
other_text_wondering    0.003037
other_text_church       0.003006
Length: 64, dtype: float32

In [144]:
clf=RandomForestClassifier(n_estimators=10000,n_jobs=-1)

clf.fit(train_features,y_train) 


imp_feats_rf=pd.Series(model.feature_importances_,index=train_features.columns)

imp_features_rf=imp_feats_rf[imp_feats_rf>0.003]



In [23]:
def val_tfidf(df,column,vectorizer):
    
    tf_val=vectorizer.transform(df[column]) 
    
    tfidf_val=pd.DataFrame(tf_val.toarray(),columns=[column+'_'+ x for x in vectorizer.get_feature_names()]) 
    
    return tfidf_val

In [24]:
def objective_function_xg(params):
    
    X=params['X']
    y=params['y']
    
    del params['X']
    del params['y']
    
    F1_Score=0
    #print(params)
    
    X_train, X_test,y_train, y_test=train_test_split (X,y, random_state=2)

    clf=xgb.XGBClassifier(**params)

    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,)

    #print('done')

    pred = clf.predict(X_test)

    F1_Score = f1_score(pred,y_test,average='macro')
    print ("SCORE:", F1_Score)
    #change the metric if you like
    return {'loss': -F1_Score/4, 'status': STATUS_OK }

In [ ]:
def objective_function_rf(params):
    
    X=params['X']
    y=params['y']
    
    del params['X']
    del params['y']
    
    F1_Score=0
    
    for i in range(4):
    
        X_train, X_test,y_train, y_test=train_test_split (X,y, random_state=i*2)
    
        
        clf=RandomForestClassifier(**params)
        
    
        evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    
    
        clf.fit(X_train, y_train,)

        pred = clf.predict(X_test)
        
        F1_Score += f1_score(pred,y_test,average='macro')
    print ("SCORE:", F1_Score)
    #change the metric if you like
    return {'loss': -F1_Score, 'status': STATUS_OK }

In [25]:
def tune_params_xg(X_train,y_train):
    
    space={'max_depth':  hp.choice('max_depth', np.arange(3, 18, dtype=int)),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform("n_estimators",50,1000,10),
        'learning_rate':hp.quniform('learning_rate',0.02,0.6,0.001),
        'X':X_train,
        'y':y_train
    }


    trials = Trials()
    best_param_xg = fmin(objective_function_xg,space=space, algo=tpe.suggest,
                      max_evals=70, trials=trials)
    
    
    return space_eval(space, best_param_xg)

In [ ]:
def tune_params_rf(X_train,y_train):
    
    space={'max_depth': hp.choice('max_depth', range(1,20)),
           'max_features': hp.choice('max_features', range(1,3)),
           'n_estimators': hp.choice('n_estimators', range(10,50)), 
           'criterion': hp.choice('criterion', ["gini", "entropy"]),
            'X':X_train,
            'y':y_train
    }




    trials = Trials()
    best_param_rf = fmin(objective_function_rf,space=space, algo=tpe.suggest,
                      max_evals=50, trials=trials)
    
    
    return space_eval(space, best_param_rf)


In [26]:
first_tfidf_t=val_tfidf(X_val,'nntp-posting-host',v1)

sec_tfidf_t=val_tfidf(X_val,'other_text',v2)

third_tfidf_t=val_tfidf(X_val,'from',v3)

fourth_tfidf_t=val_tfidf(X_val,'subject',v4)

fifth_tfidf_t=val_tfidf(X_val,'organization',v5)

sixth_tfidf_t=val_tfidf(X_val,'reply-to',v6)

seventh_tfidf_t=val_tfidf(X_val,'distribution',v7)  
  
test_features=pd.concat([first_tfidf_t,sec_tfidf_t,third_tfidf_t,fourth_tfidf_t,fifth_tfidf_t,sixth_tfidf_t,seventh_tfidf_t,X_val[['lines']].reset_index(drop=True)],axis=1)


    

In [27]:
best_params_xg=tune_params_xg(train_features[imp_features_xgb.index],y_train)

SCORE:                                                
0.00441988950276243                                   
SCORE:                                                                                
0.0376197953327197                                                                    
SCORE:                                                                                
0.00441988950276243                                                                
SCORE:                                                                             
0.00441988950276243                                                                
SCORE:                                                                             
0.00441988950276243                                                                
SCORE:                                                                             
0.00441988950276243                                                                
SCORE:                                   

In [145]:
best_params_rf=tune_params_rf(train_features[imp_features_rf.index],y_train)



In [33]:
#####xg_boost

best_model_xgb=xgb.XGBClassifier(params=best_params_xg)

best_model_xgb.fit(train_features[imp_features_xgb.index],y_train)

val_pred_xgb=best_model_xgb.predict(test_features[imp_features_xgb.index])

fval_xgb=f1_score(y_val,val_pred_xgb,average='weighted')

In [383]:
best_params_rf=best_params

In [385]:
best_params_rf.keys()

dict_keys(['X', 'criterion', 'max_depth', 'max_features', 'n_estimators', 'y'])

In [386]:
del best_params_rf['X']

In [387]:
del best_params_rf['y']

In [388]:
####rf

best_model_rf=RandomForestClassifier(**best_params_rf)

best_model_rf.fit(train_features[imp_features.index],y_train)

val_pred_rf=best_model.predict(test_features[imp_features.index])

fval_rf = f1_score(y_val,val_pred_rf,average='weighted')

*Accuracy* - F1  score weighted Average for Random forest is 48.46% for validation set(taken 20% of train data)

In [390]:
fval_rf

0.4846075326969974

Accuracy - F1 score weighted Average for Xgboost is 63.93% for validation set(taken 20% of train data)

In [253]:
fval_xgb

0.6393357322593395

In [41]:
#####TRAIN ON WHOLE DATASET

w_train_df=create_df(path_=train_path)

w_train_df=interpolate_line_lenght(w_train_df)

w_train_df.fillna('',inplace=True)


X=train_df[['nntp-posting-host', 'other_text', 'from', 'subject', 'organization',
       'lines','reply-to','distribution','all_text']]    

y=train_df['Class']

    
#X_train, X_val,y_train, y_val=train_test_split (X,y, random_state=0)

first_tfidf1,v1=tfidf(X,'nntp-posting-host')

sec_tfidf,v2=tfidf(X,'other_text')

third_tfidf,v3=tfidf(X,'from')

fourth_tfidf,v4=tfidf(X,'subject')

fifth_tfidf,v5=tfidf(X,'organization')

sixth_tfidf,v6=tfidf(X,'reply-to')

seventh_tfidf,v7=tfidf(X,'distribution')

train_features_w=pd.concat([ first_tfidf1,sec_tfidf,third_tfidf,fourth_tfidf,fifth_tfidf,sixth_tfidf,seventh_tfidf,X[['lines']].reset_index(drop=True)],axis=1)


    

/Users/shrutijain/Applications/Python/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
test_df=create_df(path_=test_path)

test_df=interpolate_line_lenght(test_df)
    
test_df.fillna('',inplace=True)

ytest=test_df['Class']

XTest=test_df[['nntp-posting-host', 'other_text', 'from', 'subject', 'organization',
       'lines','reply-to','distribution','all_text']] 


/Users/shrutijain/Applications/Python/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
first_tfidf_t=val_tfidf(XTest,'nntp-posting-host',v1)

sec_tfidf_t=val_tfidf(XTest,'other_text',v2)

third_tfidf_t=val_tfidf(XTest,'from',v3)

fourth_tfidf_t=val_tfidf(XTest,'subject',v4)

fifth_tfidf_t=val_tfidf(XTest,'organization',v5)

sixth_tfidf_t=val_tfidf(XTest,'reply-to',v6)

seventh_tfidf_t=val_tfidf(XTest,'distribution',v7)  
  
test_features_w=pd.concat([first_tfidf_t,sec_tfidf_t,third_tfidf_t,fourth_tfidf_t,fifth_tfidf_t,sixth_tfidf_t,seventh_tfidf_t,XTest[['lines']].reset_index(drop=True)],axis=1)



In [257]:
best_model_xgb=xgb.XGBClassifier(params=best_params_xg)

best_model_xgb.fit(train_features_w[imp_features.index],y)

test_pred=best_model.predict(test_features_w[imp_features.index])

ftest_xgb = f1_score(ytest,test_pred,average='weighted')


In [409]:
best_model_rf=RandomForestClassifier(**best_params)

best_model_rf.fit(train_features_w[imp_features.index],y)

test_pred=best_model.predict(test_features_w[imp_features.index])

ftest_rf = f1_score(ytest,test_pred,average='weighted')

Accuracy - F1 score weighted Average for Random forest is 38.72% for test data

In [410]:
ftest_rf

0.3872944983833138

Accuracy - F1 score weighted Average for Xgboost is 59.74% for test data

In [267]:
ftest_xgb

0.5974575780479489

In [36]:
# Deep Nueral Network

from sklearn.preprocessing import OneHotEncoder

from sklearn.decomposition import TruncatedSVD

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

import nltk

from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import confusion_matrix,f1_score,recall_score,accuracy_score,precision_score

#from Preprocessing import create_sequences,create_embedding_matrix

from tensorflow.keras.optimizers import Adam,SGD

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential

from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,LambdaCallback  

from tensorflow.keras import layers

import tensorflow as tf

from nltk.stem.snowball import SnowballStemmer

from keras.utils import plot_model

from keras.models import Model

from keras.layers import Input

from keras.layers import Dense

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

lemmatizer = nltk.stem.WordNetLemmatizer()

stemmer=SnowballStemmer('english')

from keras import losses

loss_fn = losses.SparseCategoricalCrossentropy()


In [115]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y_train_enc=tf.keras.utils.to_categorical(le.fit_transform(y_train.values))

y_test_enc=tf.keras.utils.to_categorical(le.transform(y_val.values))

In [119]:
input = layers.Input(shape=(len(train_features.columns),1))
drop20 = layers.SpatialDropout1D(0.3)(input)
conv2 = layers.Conv1D(filters=128, kernel_size=5, activation='relu')(drop20)
drop21 = layers.Dropout(0.5)(conv2)
conv22 = layers.Conv1D(filters=64, kernel_size=5, activation='relu')(drop21)
drop22 = layers.Dropout(0.5)(conv22)
pool2 = layers.MaxPooling1D(pool_size=2)(drop22)
flat2 = layers.Flatten()(pool2)
drop22 = layers.Dropout(0.3)(flat2)
out = layers.Dense(20, activation='softmax')(drop22)


In [120]:
model_nw1 = Model(input, out)
model_nw1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_nw1.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 2295, 1)]         0         
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 2295, 1)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 2291, 128)         768       
_________________________________________________________________
dropout_13 (Dropout)         (None, 2291, 128)         0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 2287, 64)          41024     
_________________________________________________________________
dropout_14 (Dropout)         (None, 2287, 64)          0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1143, 64)          0   

In [111]:
callbacks_list = [EarlyStopping(monitor='val_loss', patience=10,min_delta=0.0005),\
                  ModelCheckpoint(filepath='best_model.hdf5', monitor='val_loss',\
                                  save_best_only=True, mode='min')]

In [121]:
history = model_nw1.fit(train_features.values,y_train_enc,
                epochs=100,
                verbose=1,
                validation_data=(test_features.values, y_test_enc),
                batch_size=64,
                
                #class_weight={0:1,1:10},
                callbacks=callbacks_list)



Epoch 1/100
33/33 [==============================] - 40s 1s/step - loss: 2.8279 - accuracy: 0.1643 - val_loss: 2.3191 - val_accuracy: 0.4306
Epoch 2/100
33/33 [==============================] - 43s 1s/step - loss: 1.7347 - accuracy: 0.4711 - val_loss: 1.6374 - val_accuracy: 0.5506
Epoch 3/100
33/33 [==============================] - 42s 1s/step - loss: 1.2868 - accuracy: 0.6166 - val_loss: 1.4589 - val_accuracy: 0.5694
Epoch 4/100
33/33 [==============================] - 32s 972ms/step - loss: 1.1144 - accuracy: 0.6633 - val_loss: 1.4059 - val_accuracy: 0.5723
Epoch 5/100
33/33 [==============================] - 31s 932ms/step - loss: 1.0031 - accuracy: 0.6932 - val_loss: 1.4242 - val_accuracy: 0.5910
Epoch 6/100
33/33 [==============================] - 32s 962ms/step - loss: 0.9529 - accuracy: 0.7091 - val_loss: 1.4447 - val_accuracy: 0.6040
Epoch 7/100
33/33 [==============================] - 35s 1s/step - loss: 0.9389 - accuracy: 0.7042 - val_loss: 1.5161 - val_accuracy: 0.5954
Epoc

In [123]:
y_pred_nw1=le.inverse_transform(np.argmax(model_nw1.predict(test_features),axis=1))

Accuracy - F1 score weighted Average is 61.01% for validation data

In [125]:
f1_score(y_val,y_pred_nw1,average='weighted')


0.6101891524323161

In [133]:
pd.DataFrame(confusion_matrix(y_val,y_pred_nw1),columns=y_val.sort_values().unique(),index=y_val.sort_values().unique())

,class_1,class_10,class_11,class_12,class_13,class_14,class_15,class_16,class_17,class_18,class_19,class_2,class_20,class_3,class_4,class_5,class_6,class_7,class_8,class_9
class_1,19,0,0,0,0,1,0,0,0,2,0,0,10,0,0,0,0,0,0,0
class_10,0,23,7,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,1,1
class_11,1,2,33,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,2,0
class_12,0,0,0,26,1,0,1,0,0,0,2,1,1,0,0,2,1,0,0,0
class_13,1,0,0,3,17,0,1,0,0,0,0,0,0,1,2,2,2,3,2,1
class_14,0,0,2,0,1,26,1,1,0,0,0,2,0,2,1,0,1,1,1,1
class_15,0,2,0,0,4,0,27,0,0,0,1,0,0,0,1,0,1,0,0,1
class_16,1,0,0,0,2,1,0,18,0,0,1,2,7,0,0,0,0,0,0,0
class_17,0,0,1,2,1,0,0,1,19,0,2,0,0,0,0,0,1,1,2,2
class_18,0,0,0,0,0,1,1,0,1,22,4,0,0,1,0,0,0,0,1,0


In [139]:
####for whole data set

y_train_enc=tf.keras.utils.to_categorical(le.fit_transform(y.values))

y_test_enc=tf.keras.utils.to_categorical(le.transform(ytest.values))

In [140]:
input = layers.Input(shape=(len(train_features_w.columns),1))
drop20 = layers.SpatialDropout1D(0.3)(input)
conv2 = layers.Conv1D(filters=128, kernel_size=5, activation='relu')(drop20)
drop21 = layers.Dropout(0.5)(conv2)
conv22 = layers.Conv1D(filters=64, kernel_size=5, activation='relu')(drop21)
drop22 = layers.Dropout(0.5)(conv22)
pool2 = layers.MaxPooling1D(pool_size=2)(drop22)
flat2 = layers.Flatten()(pool2)
drop22 = layers.Dropout(0.3)(flat2)
dense1 = layers.Dense(32)(drop22)
#flat2 = layers.Flatten()(pool2)
drop22 = layers.Dropout(0.3)(dense1)
out = layers.Dense(20, activation='softmax')(drop22)


In [141]:
model_nw2 = Model(input, out)
model_nw2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_nw2.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 2948, 1)]         0         
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 2948, 1)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 2944, 128)         768       
_________________________________________________________________
dropout_16 (Dropout)         (None, 2944, 128)         0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 2940, 64)          41024     
_________________________________________________________________
dropout_17 (Dropout)         (None, 2940, 64)          0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1470, 64)          0   

In [142]:
callbacks_list = [EarlyStopping(monitor='val_loss', patience=10,min_delta=0.0005),\
                  ModelCheckpoint(filepath='best_model.hdf5', monitor='val_loss',\
                                  save_best_only=True, mode='min')]

In [143]:
history = model_nw2.fit(train_features_w.values,y_train_enc,
                epochs=20,
                verbose=1,
                validation_data=(test_features_w.values, y_test_enc),
                batch_size=64,
                
                #class_weight={0:1,1:10},
                callbacks=callbacks_list)



Epoch 1/20
44/44 [==============================] - 84s 2s/step - loss: 2.6157 - accuracy: 0.2171 - val_loss: 1.9208 - val_accuracy: 0.4759
Epoch 2/20
44/44 [==============================] - 96s 2s/step - loss: 1.6859 - accuracy: 0.4928 - val_loss: 1.6495 - val_accuracy: 0.5416
Epoch 3/20
44/44 [==============================] - 82s 2s/step - loss: 1.3666 - accuracy: 0.5809 - val_loss: 1.4641 - val_accuracy: 0.5753
Epoch 4/20
44/44 [==============================] - 80s 2s/step - loss: 1.1681 - accuracy: 0.6405 - val_loss: 1.4454 - val_accuracy: 0.5763
Epoch 5/20
44/44 [==============================] - 82s 2s/step - loss: 1.1469 - accuracy: 0.6413 - val_loss: 1.4557 - val_accuracy: 0.5820
Epoch 6/20
44/44 [==============================] - 81s 2s/step - loss: 1.0345 - accuracy: 0.6770 - val_loss: 1.4881 - val_accuracy: 0.5758
Epoch 7/20
44/44 [==============================] - 61s 1s/step - loss: 1.0101 - accuracy: 0.6835 - val_loss: 1.4867 - val_accuracy: 0.5872
Epoch 8/20
44/44 [==

In [147]:
y_pred=le.inverse_transform(np.argmax(model_nw2.predict(test_features_w),axis=1))

In [148]:
fval_nw2 = f1_score(ytest,y_pred,average='weighted')

Accuracy - F1 score weighted Average is 58.13% for test data

In [149]:
fval_nw2

0.5738420100594281

In [151]:
pd.DataFrame(confusion_matrix(ytest,y_pred),columns=ytest.sort_values().unique(),index=ytest.sort_values().unique())

,class_1,class_10,class_11,class_12,class_13,class_14,class_15,class_16,class_17,class_18,class_19,class_2,class_20,class_3,class_4,class_5,class_6,class_7,class_8,class_9
class_1,34,0,1,1,1,3,3,8,1,2,3,3,21,0,0,1,0,2,3,1
class_10,2,67,23,0,2,1,2,1,0,0,5,2,2,0,0,0,0,2,4,1
class_11,1,6,83,0,0,0,2,0,0,0,0,1,1,1,0,0,1,0,1,0
class_12,2,1,0,66,2,1,5,1,4,0,8,4,1,5,2,1,0,2,1,0
class_13,2,0,3,1,34,1,3,0,0,1,4,6,1,7,8,6,4,1,10,4
class_14,3,1,2,2,3,47,6,5,0,1,8,1,2,3,2,0,1,2,1,1
class_15,2,0,4,1,1,3,74,1,2,0,2,2,2,1,0,1,1,1,5,2
class_16,4,0,0,0,1,3,3,35,0,0,2,0,45,2,0,0,1,0,1,0
class_17,1,1,2,4,0,1,1,2,53,2,9,0,5,0,1,1,1,2,0,1
class_18,8,1,1,0,1,0,3,1,3,63,5,0,13,0,2,0,0,0,1,2


In [152]:
#from gensim.models import Doc2Vec
from gensim.models.keyedvectors import KeyedVectors

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, errors = 'ignore', encoding="utf8") as f:
        for line in f:
            
            word, *vector = line.split()
            
            if word in word_index:
                
                idx = word_index[word]
                
                try:
                    embedding_matrix[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]
                    
                except ValueError:
                    continue
                    
    return embedding_matrix


/Users/shrutijain/Applications/Python/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [153]:
def create_sequences(text_series,max_=250,tokenizer=None):
    '''
        Creates sequences from text data
    '''
    
    if not tokenizer:
        
        tokenizer = Tokenizer(num_words=5000)
        
        tokenizer.fit_on_texts(text_series)
        
        
        
    return pad_sequences(tokenizer.texts_to_sequences(text_series),\
                         padding='post',maxlen=max_),tokenizer
                         

In [154]:
def get_functional_column(X1,X2,X_sub,X_org,vocab_size,embedding_dim,embedding_matrix,maxlen,optimizer,\
              layer_s={'conv':{0:(64,3,3),1:(64,3,2)},'lstm':[(16,0.25)
                            ],
                               'drop':[0.2,0.2],'dense':[32,16]},svocab=1000,\
                  ovocab=1000,smax=25,omax=10
              ):
#######################################################################################################    
    visible = layers.Input(shape=(X1.shape[1],))
    
    embed=layers.Embedding(vocab_size+1, embedding_dim, 
                               weights=[embedding_matrix], 
                               input_length=maxlen, 
                               trainable=False)(visible)
    
    last=[embed]
    
    for layer in layer_s['conv'].keys():
       
       last.append(layers.Conv1D(layer_s['conv'][layer][0],layer_s['conv'][layer][1]\
                               ,strides=layer_s['conv'][layer][2],activation='relu',padding='valid')(last[-1]))

    
    for n_layer in range(len(layer_s['lstm'])):
        
        if n_layer==len(layer_s['lstm'])-1:
            
            last.append(layers.LSTM(layer_s['lstm'][n_layer][0], return_sequences=False)(last[-1]))
            
            last.append(layers.Dropout(layer_s['lstm'][n_layer][1])(last[-1]))
            
        else:
            
            last.append(layers.LSTM(layer_s['lstm'][n_layer][0], return_sequences=True)(last[-1]))
            
            last.append(layers.Dropout(layer_s['lstm'][n_layer][1])(last[-1]))
            
    last.append(layers.BatchNormalization()(last[-1]))
    
    sub_input= layers.Input(shape=(X_sub.shape[1],))
    
    embed1=layers.Embedding(svocab+1, embedding_dim, 
                               weights=[embedding_matrix], 
                               input_length=25, 
                               trainable=False)(sub_input)
#######################################################################################################    
    sub_conv=layers.Conv1D(8,2,strides=1,activation='relu',padding='valid')(embed1)
    
    lstm=layers.LSTM(8, return_sequences=False)(sub_conv)
    
    lstm_drp=layers.Dropout(0.2)(lstm)
    
    lstm_drp=layers.BatchNormalization()(lstm_drp)
    
    sub0 = layers.Dense(16, activation='relu')(lstm_drp)
    
    sub0b=layers.BatchNormalization()(sub0)
    
    sub0b=layers.Dropout(0.1)(sub0b)
    
 #######################################################################################################  
    org_input= layers.Input(shape=(X_org.shape[1],))
    
    embed1=layers.Embedding(ovocab+1, embedding_dim, 
                               weights=[embedding_matrix], 
                               input_length=10, 
                               trainable=False)(org_input)
    
    org_conv=layers.Conv1D(8,2,strides=1,activation='relu',padding='valid')(embed1)
    
    lstm=layers.LSTM(8, return_sequences=False)(org_conv)
    
    lstm_drp=layers.Dropout(0.2)(lstm)
    
    lstm_drp=layers.BatchNormalization()(lstm_drp)
    
    org0 = layers.Dense(16, activation='relu')(lstm_drp)
    
    org0b=layers.BatchNormalization()(org0)
    
    org0b=layers.Dropout(0.1)(org0b)
    
 #######################################################################################################   
            
    other_input = layers.Input(shape=(X2.shape[1], ))
    
    hidden0 = layers.Dense(16, activation='relu')(other_input)
    
    hidden0b=layers.BatchNormalization()(hidden0)
    
    hidden0b=layers.Dropout(0.1)(hidden0b)
    
    hidden1 = layers.Dense(16, activation='relu')(hidden0b)
    
    hidden1b=layers.BatchNormalization()(hidden1)

#######################################################################################################
    
    merge = layers.Concatenate()([last[-1], hidden1b, org0b, sub0b])
    
    hidden2 = layers.Dense(128, activation='relu')(merge)
    
    hidden2b=layers.BatchNormalization()(hidden2)
    
    hidden2b=layers.Dropout(0.2)(hidden2b)
    
    hidden3 = layers.Dense(64, activation='relu')(hidden2b)
    
    output=layers.Dense(20, activation='softmax')(hidden3)
    
    
    
    model = Model(inputs=[visible,other_input,sub_input,org_input], outputs=output)
    
    model.compile(optimizer=optimizer,
                  #loss=Custom_Loss(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    

#summarize layers
    print(model.summary())
    
    return model

In [155]:
def get_model(vocab_size,embedding_dim,embedding_matrix,maxlen,optimizer,\
              layer_s={'conv':{0:(64,3),1:(64,3)},'lstm':[(64,0.25)
                                    ,(32,0.25)
                            ],
                               'drop':[0.5,0.5],'dense':[32,16]},
              ):
    
    model = Sequential()
    
    model.add(layers.Embedding(vocab_size+1, embedding_dim, 
                               weights=[embedding_matrix], 
                               input_length=maxlen, 
                               trainable=False))
    
    for layer in layer_s['conv'].keys():
        
        model.add(layers.Conv1D(layer_s['conv'][layer][0],layer_s['conv'][layer][1]\
                                ,strides=layer_s['conv'][layer][2],activation='relu',padding='valid'))
            
    
    
        
    #model.add(layers.BatchNormalization())
        
    
    #model.add(layers.Dropout(layer_s['drop'][0]))
        
    
    for n_layer in range(len(layer_s['lstm'])):
        
        if n_layer==len(layer_s['lstm'])-1:
            
            model.add(layers.LSTM(layer_s['lstm'][n_layer][0], return_sequences=False))
            
            model.add(layers.Dropout(layer_s['lstm'][n_layer][1]))
            
        else:
            
            model.add(layers.LSTM(layer_s['lstm'][n_layer][0], return_sequences=True))
            
            model.add(layers.Dropout(layer_s['lstm'][n_layer][1]))
            
        
    model.add(layers.BatchNormalization())
        
        
    for n_ in range(len(layer_s['dense'])):
        
        model.add(layers.Dense(layer_s['dense'][n_],activation='relu'))
        
        model.add(layers.Dropout(layer_s['drop'][n_]))
        
    
    model.add(layers.Dense(20, activation='softmax')) # for twenty y classes
    
    
    model.compile(optimizer=optimizer,
                  #loss=Custom_Loss(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()
    
    return model


In [156]:
bestLoss = 1000000000000000000
bestWeights = None

In [157]:
optimizer=Adam(lr=0.05)

embedding_dim=100

maxlen=500

In [158]:
train_sequence,tkn = create_sequences(X_train.other_text.values,max_=maxlen)

test_sequence,tkn = create_sequences(X_val.other_text.values,tokenizer=tkn,max_=maxlen)

vocab_size=len(tkn.word_index)


In [159]:
train_sequence_sub,stkn = create_sequences(X_train.subject.values,max_=25)

test_sequence_sub,stkn = create_sequences(X_val.subject,tokenizer=tkn,max_=25)

svocab_size=len(stkn.word_index)


train_sequence_org,otkn = create_sequences(X_train.organization.values,max_=10)

test_sequence_org,otkn = create_sequences(X_val.organization.values,tokenizer=tkn,max_=10)

ovocab_size=len(stkn.word_index)


embedding_matrix = create_embedding_matrix('/Users/shrutijain/Downloads/glove.6B.100d.txt',tkn.word_index, embedding_dim)


In [169]:
model=get_functional_column(train_sequence,train_features,\
                            train_sequence_sub,train_sequence_org,\
                            vocab_size,embedding_dim,embedding_matrix,maxlen,optimizer,\
                            svocab=svocab_size,ovocab=ovocab_size,smax=25,omax=10
          )


 

callbacks_list = [EarlyStopping(monitor='val_loss', patience=10,min_delta=0.0005),\
                  ModelCheckpoint(filepath='best_model.hdf5', monitor='val_loss',\
                                  save_best_only=True, mode='min')]

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 10, 100)      4534800     input_17[0][0]                   
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 25, 100)      4534800     input_16[0][0]                   
____________________________________________________________________________________________

In [170]:
le=LabelEncoder()    
    
y_train_enc=tf.keras.utils.to_categorical(le.fit_transform(y_train.values))

y_test_enc=tf.keras.utils.to_categorical(le.transform(y_val.values))

#%%
X_train_new=[train_sequence,train_features,\
                            train_sequence_sub,train_sequence_org]
    
X_test_new=[test_sequence,test_features,\
                            test_sequence_sub,test_sequence_org]
    
    
history = model.fit(X_train_new,y_train_enc,
                epochs=50,
                verbose=1,
                validation_data=(X_test_new, y_test_enc),
                batch_size=128,
                
                #class_weight={0:1,1:10},
                callbacks=callbacks_list)


Epoch 1/50
17/17 [==============================] - 4s 254ms/step - loss: 3.6403 - accuracy: 0.0568 - val_loss: 212.0939 - val_accuracy: 0.0621
Epoch 2/50
17/17 [==============================] - 3s 170ms/step - loss: 3.0628 - accuracy: 0.0597 - val_loss: 10.7983 - val_accuracy: 0.0607
Epoch 3/50
17/17 [==============================] - 3s 173ms/step - loss: 3.0197 - accuracy: 0.0530 - val_loss: 8.9520 - val_accuracy: 0.0564
Epoch 4/50
17/17 [==============================] - 3s 170ms/step - loss: 3.0103 - accuracy: 0.0491 - val_loss: 3.0796 - val_accuracy: 0.0578
Epoch 5/50
17/17 [==============================] - 3s 173ms/step - loss: 3.0023 - accuracy: 0.0564 - val_loss: 3.0239 - val_accuracy: 0.0592
Epoch 6/50
17/17 [==============================] - 3s 158ms/step - loss: 3.0013 - accuracy: 0.0607 - val_loss: 3.0259 - val_accuracy: 0.0592
Epoch 7/50
17/17 [==============================] - 3s 171ms/step - loss: 2.9984 - accuracy: 0.0539 - val_loss: 2.9959 - val_accuracy: 0.0578
Epo